In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import ListedColormap
from IPython.display import display, Image

# Function to simulate the Spatial Prisoner's Dilemma on a grid without torus boundary conditions
def spatial_pd_8_nn_no_torus(b, p, self_interaction, gens, n, flag, print_lattice, print_fc, limit):
    # Initialize parameters
    nn = n * n  # Total number of cells (n x n grid)
    R, S, T, P = 1, 0, b, 0  # Payoff values for Prisoner's Dilemma
    payoff_matrix = np.array([[R, S], [T, P]])  # 2x2 payoff matrix for the game

    # Define the color scheme for visualization: Blue for no change, Green for C to C, Yellow for D to D, Red for D to C
    color_matrix = ListedColormap([[0, 0, 1], [0, 1, 0], [1, 1, 0], [1, 0, 0]])

    # Transition matrix to track changes in strategies (1: C to C, 2: D to D, 3: D to C, 4: C to D)
    transitions = np.array([[1, 3], [2, 4]])

    # Array to track the fraction of sites occupied by cooperators (C) over generations
    fc = np.zeros(gens)

    # Initialize payoff and lattice matrices
    payoff = np.zeros((n, n))  # Payoff matrix for each cell
    new_lattice = np.zeros((n, n))  # New strategy matrix after each round
    transition_matrix = np.zeros((n, n, gens))  # To track strategy transitions across generations
    lattice = np.ones((n, n))  # Initialize lattice with all cells as cooperators (C)

    # Initial condition: place a defector (D) in the center or randomly populate based on flag
    if flag == 1:
        lattice[n // 2, n // 2] = 2  # Place a single defector in the center
    else:
        lattice2 = np.random.rand(n, n)  # Generate a random lattice
        lattice[lattice2 < p] = 2  # Populate lattice with defectors (D) based on probability p

    # Main loop over generations
    for iter in range(gens):
        # Calculate the payoff for each cell based on neighboring cells' strategies
        for i in range(n):
            for j in range(n):
                temp_payoff = 0  # Temporary variable to sum up payoffs for cell (i, j)
                for k in range(-1, 2):
                    for h in range(-1, 2):
                        # Skip self-interaction if specified
                        if self_interaction == 0 and k == 0 and h == 0:
                            continue
                        # Calculate the neighbor's coordinates
                        a, b = i + k, j + h
                        # Skip if the neighbor is out of bounds
                        if a < 0 or a >= n or b < 0 or b >= n:
                            continue
                        # Add the payoff from interaction with the neighbor
                        temp_payoff += payoff_matrix[int(lattice[i, j]) - 1, int(lattice[a, b]) - 1]
                payoff[i, j] = temp_payoff  # Update the payoff matrix for cell (i, j)

        # Update each cell's strategy based on the highest neighboring payoff
        for i in range(n):
            for j in range(n):
                max_payoff = payoff[i, j]  # Start with the cell's own payoff
                strategy = lattice[i, j]  # Start with the cell's own strategy
                for k in range(-1, 2):
                    for h in range(-1, 2):
                        # Skip self-interaction if specified
                        if self_interaction == 0 and k == 0 and h == 0:
                            continue
                        # Calculate the neighbor's coordinates
                        a, b = i + k, j + h
                        # Skip if the neighbor is out of bounds
                        if a < 0 or a >= n or b < 0 or b >= n:
                            continue
                        # If a neighbor has a higher payoff, adopt their strategy
                        if payoff[a, b] > max_payoff:
                            max_payoff = payoff[a, b]
                            strategy = lattice[a, b]
                new_lattice[i, j] = strategy  # Update the new strategy for cell (i, j)

        # Update the lattice and record the transitions
        for i in range(n):
            for j in range(n):
                # Record the transition type (e.g., C to D, D to C)
                transition_matrix[i, j, iter] = transitions[int(new_lattice[i, j]) - 1, int(lattice[i, j]) - 1]
                lattice[i, j] = new_lattice[i, j]  # Update the lattice for the next generation

        # Calculate the fraction of cells occupied by cooperators (C)
        fc[iter] = np.sum((transition_matrix[:, :, iter] == 1) | (transition_matrix[:, :, iter] == 3)) / nn

    # Plot lattice evolution as an animation if print_lattice is 1
    if print_lattice == 1:
        fig, ax = plt.subplots(figsize=(6, 6))  # Set up the figure with a larger size for better visibility
        mat = ax.matshow(transition_matrix[:, :, 0], cmap=color_matrix)  # Initialize the plot with the first frame

        # Adjust layout to provide more space for the title
        plt.subplots_adjust(top=0.85)  # Adjust the top margin to ensure the title is fully visible

        # Set the title with parameters displayed and add padding to avoid cutting off the text
        ax.set_title(f'Spatial Prisoner\'s Dilemma\nb={b}, p={p}, self_interaction={self_interaction}, gens={gens}, n={n}, limit={limit}', pad=20)

        # Function to update the animation for each frame
        def update(frame):
            mat.set_data(transition_matrix[:, :, frame])  # Update the lattice display for the current frame
            # Update the title to reflect the current round and parameters
            ax.set_title(f'Round: {frame + 1}\n'
                         f'b={b}, p={p}, self_interaction={self_interaction}, gens={gens}, n={n}, limit={limit}', pad=20)
            return [mat]

        # Create the animation, updating every 200 milliseconds
        ani = animation.FuncAnimation(fig, update, frames=gens, interval=200, blit=True)
        ani.save('animation.gif', writer='pillow')  # Save the animation as a GIF

        # Display the GIF using IPython.display.Image
        display(Image(filename='animation.gif'))

    # Plot the fraction of cooperators over time if print_fc is 1
    if print_fc == 1:
        plt.figure()  # Create a new figure for the fraction plot
        # Plot the fraction of sites occupied by cooperators over generations
        plt.plot(range(1, gens + 1), fc, '-p', color='purple', markersize=5, markeredgewidth=0.5)
        plt.xlabel('Round')  # Label the x-axis
        plt.ylabel('Fraction of sites occupied by C')  # Label the y-axis
        # Set the title with parameters displayed and add padding to avoid cutting off the text
        plt.title('SPATIAL PRISONER\'S DILEMMA\n'
                  f'b={b}, p={p}, self_interaction={self_interaction}, gens={gens}, n={n}, limit={limit}', pad=20)
        plt.grid(True)  # Add a grid to the plot for better readability
        if limit > 0:
            # If a limit is specified, draw a horizontal line at the limit
            plt.axhline(y=limit, color='k', linestyle='--')
            plt.text(gens, limit, str(limit))  # Label the limit line
        plt.show()  # Display the plot
